<a href="https://colab.research.google.com/github/ethvedbitdesjan/SummerResearch/blob/main/FunnySVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('content/')

Drive already mounted at content/; to attempt to forcibly remount, call drive.mount("content/", force_remount=True).


In [2]:
%cd /content/content/MyDrive/SummerResearch

/content/content/MyDrive/SummerResearch


In [3]:
!pip install cuml-cu11 --extra-index-url=https://pypi.nvidia.com
!pip install cupy-cuda11x

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [4]:
import cuml
import cupy as cp

In [5]:
# Critical imports
import os
import numpy as np
import pandas as pd
from PIL import Image
import random
import cv2
import copy

In [6]:
import torch
import torchvision
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim
from torch.optim import lr_scheduler

In [7]:
funny_set = set()
not_funny_set = set()
with open('labelled_data/funny_combined1.csv', 'r') as f:
    lines = f.readlines()
    for line in lines:
        funny_set.add(line.strip())
with open('labelled_data/not_funny_combined1.csv', 'r') as f:
    lines = f.readlines()
    for line in lines:
        not_funny_set.add(line.strip())

In [8]:
train_data = []
train_funny_data = []
train_not_funny_data = []
count = 0
added = set()
for file in os.listdir('Funny_Originals_Final'):
  img_array = Image.open(os.path.join('Funny_Originals_Final', file))
  if file in funny_set:
    added.add(file)
    train_funny_data.append((img_array, 1))
  if file in not_funny_set:
    added.add(file)
    train_not_funny_data.append((img_array, 0))
  count += 1
for file in os.listdir('Non-Funny_Modified_Final'):
  img_array = Image.open(os.path.join('Non-Funny_Modified_Final', file))
  if file in funny_set:
    added.add(file)
    train_funny_data.append((img_array, 1))
  if file in not_funny_set:
    added.add(file)
    train_not_funny_data.append((img_array, 0))
random.seed()

In [9]:
train_data = train_funny_data + train_not_funny_data

In [10]:
random.shuffle(train_data)

In [11]:
class FunnyNotFunnyDataset(Dataset):
    def __init__(self, data, root_dir=None, transform=None):
        self.data = data
        self.root_dir = root_dir
        self.transform = transform
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        image = self.data[index][0]
        if self.transform:
          image = self.transform(image)
        label = self.data[index][1]
        label_tensor = torch.zeros(1)
        if label == 1:
          label_tensor[0] = 1
        return {'image_data':image, 'label':label_tensor}

In [12]:
class ResNetAdded(torch.nn.Module):
  def __init__(self, resnet50=None, train_full=True):
    super(ResNetAdded, self).__init__()
    if resnet50:
      self.resnet50 = resnet50
    else:
      self.resnet50 = models.resnet50(pretrained=True)
    self.resnet50.fc = torch.nn.Identity()
    self.classifier = torch.nn.Linear(2048, 1)
    if not train_full:
      for param in self.resnet50.parameters():
        param.requires_grad = False

  def forward(self, x):
    """
    x: A batch of images.

    Returns: A tensor of predictions.
    """

    x = self.resnet50(x)
    preds = self.classifier(x)
    return preds

  def feature_extractor(self, x):
    features = self.resnet50(x)
    return features

In [13]:
idx_train_split = int(0.6 * len(train_data))
idx_test_split = int(0.7*len(train_data))
model = ResNetAdded()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
IMAGE_SIZE = 224
NUM_CLASSES = 1103
BATCH_SIZE = 1
device = 'cuda' if torch.cuda.is_available() else 'cpu'
IMG_MEAN = [0.485, 0.456, 0.406]
IMG_STD = [0.229, 0.224, 0.225]
train_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(IMG_MEAN, IMG_STD)
])
test_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(IMG_MEAN, IMG_STD)
])
train_dataset = FunnyNotFunnyDataset(train_data[:idx_test_split], transform = train_transform)
test_dataset = FunnyNotFunnyDataset(train_data[idx_test_split:] ,transform = test_transform)
train_dataloader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle=True)
model.to(device)

ResNetAdded(
  (resnet50): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
     

In [15]:
import gc
gc.collect()

0

In [16]:
model.load_state_dict(torch.load('best_model_full1.bin'))
model.to(device)

ResNetAdded(
  (resnet50): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
     

In [17]:
funny_num = 0
for data in train_dataloader:
  targets = data["label"].to(device)
  funny_num += int(torch.sum(targets.flatten()))
funny_num

778

In [18]:
model.eval()
train_svm_x = []
train_svm_y = []
not_funny_num = 0
for data in train_dataloader:
  gc.collect()
  torch.cuda.empty_cache()
  inputs = data["image_data"].to(device)
  targets = data["label"].to(device)

  inputs = inputs.type(torch.cuda.FloatTensor)
  targets = targets.type(torch.cuda.FloatTensor)
  features = model.feature_extractor(inputs)
  features = features.flatten()
  targets = targets.flatten()
  if int(torch.sum(targets.flatten())) == 0:
    not_funny_num += 1
    if not_funny_num > funny_num:
      continue
  #print(features.shape, targets.shape)
  train_svm_x.append(cp.from_dlpack(features.detach()))
  del features
  train_svm_y.append(cp.asarray(targets))

In [19]:
gc.collect()

0

In [20]:
train_svm_x = cp.array(train_svm_x)
train_svm_y  = cp.array(train_svm_y)

In [21]:
cp.save('train_svm_x1.npy', train_svm_x)
cp.save('train_svm_y1.npy', train_svm_y)

In [17]:
funny_num = 0
for data in test_dataloader:
  targets = data["label"].to(device)
  funny_num += int(torch.sum(targets.flatten()))
funny_num

346

In [18]:
test_svm_x = []
test_svm_y = []
not_funny_num = 0
for data in test_dataloader:
  gc.collect()
  torch.cuda.empty_cache()
  inputs = data["image_data"].to(device)
  targets = data["label"].to(device)

  inputs = inputs.type(torch.cuda.FloatTensor)
  targets = targets.type(torch.cuda.FloatTensor)
  features = model.feature_extractor(inputs)
  features = features.flatten()
  targets = targets.flatten()
  if int(torch.sum(targets.flatten())) == 0:
    not_funny_num += 1
    if not_funny_num > funny_num:
      continue
  #print(features.shape, targets.shape)
  test_svm_x.append(cp.from_dlpack(features.detach()))
  test_svm_y.append(cp.asarray(targets))

In [19]:
test_svm_x, test_svm_y = cp.array(test_svm_x), cp.array(test_svm_y)

In [20]:
from cuml.svm import SVC
svm = SVC()

In [21]:
cp.save('test_svm_x1.npy', test_svm_x)
cp.save('test_svm_y1.npy', test_svm_y)

In [22]:
train_svm_x = cp.load('train_svm_x.npy')
train_svm_y = cp.load('train_svm_y.npy')

In [23]:
svm.fit(train_svm_x, train_svm_y)

SVC()

In [26]:
test_svm_x = cp.load('test_svm_x.npy')
test_svm_y = cp.load('test_svm_y.npy')
test_svm_x1 = cp.load('test_svm_x1.npy')
test_svm_y1 = cp.load('test_svm_y1.npy')

In [27]:
cp.sum(test_svm_y), test_svm_y.shape, cp.sum(test_svm_y1), test_svm_y1.shape

(array(330., dtype=float32), (808, 1), array(346., dtype=float32), (692, 1))

In [28]:
from cuml.metrics import accuracy_score, confusion_matrix
y_pred = svm.predict(test_svm_x)
print(accuracy_score(y_pred, test_svm_y))
pd.DataFrame(confusion_matrix(y_pred.flatten().astype(int), test_svm_y.flatten().astype(int)).get(), columns=['Test Not Funny', 'Test Funny'], index=['Pred Not Funny', 'Pred Funny'])

0.5891088843345642


,Test Not Funny,Test Funny
Pred Not Funny,475,329
Pred Funny,3,1


In [29]:
y_pred1 = svm.predict(test_svm_x1)
print(accuracy_score(y_pred1, test_svm_y1))
pd.DataFrame(confusion_matrix(y_pred1.flatten().astype(int), test_svm_y1.flatten().astype(int)).get(), columns=['Test1 Not Funny', 'Test1 Funny'], index=['Pred Not Funny', 'Pred Funny'])

0.5


,Test1 Not Funny,Test1 Funny
Pred Not Funny,346,346
Pred Funny,0,0


In [30]:
train_svm_x1 = cp.load('train_svm_x1.npy')
train_svm_y1 = cp.load('train_svm_y1.npy')
svm1 = SVC()
svm1.fit(train_svm_x1, train_svm_y1)

SVC()

In [31]:
y_pred = svm1.predict(test_svm_x)
print(accuracy_score(y_pred, test_svm_y))
pd.DataFrame(confusion_matrix(y_pred.flatten().astype(int), test_svm_y.flatten().astype(int)).get(), columns=['Test Not Funny', 'Test Funny'], index=['Pred1 Not Funny', 'Pred1 Funny'])

0.5878713130950928


,Test Not Funny,Test Funny
Pred1 Not Funny,444,299
Pred1 Funny,34,31


In [32]:
y_pred1 = svm1.predict(test_svm_x1)
print(accuracy_score(y_pred1, test_svm_y1))
pd.DataFrame(confusion_matrix(y_pred1.flatten().astype(int), test_svm_y1.flatten().astype(int)).get(), columns=['Test1 Not Funny', 'Test1 Funny'], index=['Pred1 Not Funny', 'Pred1 Funny'])

0.5


,Test1 Not Funny,Test1 Funny
Pred1 Not Funny,346,346
Pred1 Funny,0,0


In [33]:
train_pred = svm.predict(train_svm_x)
print(accuracy_score(train_pred, train_svm_y))
pd.DataFrame(confusion_matrix(train_pred.flatten().astype(int), train_svm_y.flatten().astype(int)).get(), columns=['Train Not Funny', 'Train Funny'], index=['Pred Not Funny', 'Pred Funny'])

0.6359102129936218


,Train Not Funny,Train Funny
Pred Not Funny,444,221
Pred Funny,363,576


In [34]:
train_pred1 = svm.predict(train_svm_x1)
print(accuracy_score(train_pred1, train_svm_y1))
pd.DataFrame(confusion_matrix(train_pred1.flatten().astype(int), train_svm_y1.flatten().astype(int)).get(), columns=['Train1 Not Funny', 'Train1 Funny'], index=['Pred Not Funny', 'Pred Funny'])

0.5


,Train1 Not Funny,Train1 Funny
Pred Not Funny,778,778
Pred Funny,0,0


In [35]:
train_pred = svm1.predict(train_svm_x)
print(accuracy_score(train_pred, train_svm_y))
pd.DataFrame(confusion_matrix(train_pred.flatten().astype(int), train_svm_y.flatten().astype(int)).get(), columns=['Train Not Funny', 'Train Funny'], index=['Pred1 Not Funny', 'Pred1 Funny'])

0.49501246213912964


,Train Not Funny,Train Funny
Pred1 Not Funny,34,37
Pred1 Funny,773,760


In [36]:
train_pred1 = svm1.predict(train_svm_x1)
print(accuracy_score(train_pred1, train_svm_y1))
pd.DataFrame(confusion_matrix(train_pred1.flatten().astype(int), train_svm_y1.flatten().astype(int)).get(), columns=['Train1 Not Funny', 'Train1 Funny'], index=['Pred1 Not Funny', 'Pred1 Funny'])

0.8656812310218811


,Train1 Not Funny,Train1 Funny
Pred1 Not Funny,658,89
Pred1 Funny,120,689
